In [2]:
pd.set_option('display.height',1000)
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)
pd.set_option('display.width',1000)

In [1]:
from sklearn.metrics import log_loss
import pandas as pd
import numpy as np
from sklearn.ensemble import BaggingClassifier
import lightgbm as lgb
import gc
from sklearn.pipeline import  Pipeline
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import RidgeClassifierCV,LogisticRegression
import numpy as np
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline, make_union
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import FunctionTransformer
from copy import copy
import pickle
from sklearn.model_selection import train_test_split
import copy as cp
import gc
from sklearn.model_selection import KFold,StratifiedKFold

In [3]:
#读数据
gc.collect()
user_feature = pickle.load(open('../../data/processed/user_feature.pkl','rb+'))
shop_feature = pickle.load(open("../../data/processed/shop_feature.pkl","rb+"))
item_feature = pickle.load(open('../../data/processed/item_feature.pkl',"rb+"))
context_feature = pickle.load(open('../../data/processed/context_feature.pkl',"rb+"))
leak_feature = pickle.load(open("../../data/processed/leak_feature.pkl","rb+"))
current_feature = pickle.load(open("../../data/processed/current_feature.pkl","rb+"))
global_feature = pickle.load(open("../../data/processed/global_feature.pkl","rb+"))
history_feature = pickle.load(open("../../data/processed/preday_history_feature.pkl","rb+"))
history_31_6_feature = pickle.load(open("../../data/processed/history_31_6_cvr.pkl","rb+"))
diff_feature = pickle.load(open("../../data/processed/diff_feature.pkl","rb+"))
magic_feature = pickle.load(open("../../data/processed/magric_feature.pkl","rb+"))

flow_feature = pickle.load(open("../../data/processed/flow_feature.pkl","rb+"))

word2vec = pickle.load(open('../../data/processed/all_data_with_history7_1_tw2v.pkl','rb+'))


#user_feature.pop("is_trade")
shop_feature.pop("is_trade")
item_feature.pop("is_trade")
context_feature.pop("is_trade")
leak_feature.pop("is_trade")
current_feature.pop("is_trade")
global_feature.pop("is_trade")
history_feature.pop("is_trade")
history_31_6_feature.pop("is_trade")
diff_feature.pop("is_trade")
magic_feature.pop("is_trade")
flow_feature.pop("is_trade")



user_feature = user_feature.reset_index(drop=True)
shop_feature = shop_feature.reset_index(drop=True)
item_feature = item_feature.reset_index(drop=True)
context_feature = context_feature.reset_index(drop=True)
leak_feature = leak_feature.reset_index(drop=True)
current_feature = current_feature.reset_index(drop=True)
global_feature = global_feature.reset_index(drop=True)
history_feature = history_feature.reset_index(drop=True)
history_31_6_feature = history_31_6_feature.reset_index(drop=True)
diff_feature = diff_feature.reset_index(drop=True)
magic_feature = magic_feature.reset_index(drop=True)
flow_feature = flow_feature.reset_index(drop=True)
word2vec = word2vec.reset_index(drop=True)

del diff_feature["shop_id"]
#
del word2vec["is_trade"]
del word2vec["user_id"]
del word2vec["item_id"]
#
del context_feature["hour"]

df = pd.concat([user_feature,
               shop_feature,
               item_feature,
               context_feature,
               leak_feature,
               current_feature,
               global_feature,
               history_feature,
               history_31_6_feature,
               diff_feature,
               magic_feature,
               flow_feature,
               word2vec,
               ],axis=1)
#df = df.loc[(df["context_timestamp"] >= "20180907 01:00:00")]

In [1]:
#特征集合
base_features = ['user_age_level', 'user_occupation_id', 'user_star_level', 'user_gender_id', 'shop_review_num_level', 
                 'shop_review_positive_rate', 'shop_star_level', 'shop_score_service', 'shop_score_delivery', 'shop_score_description',
                 'item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level', 'negative_seconds_gap',
                 'negative_visited_item', 'negative_visited_shop', 'is_last_today', 'count_global_user_id_item_id', 'positive_visited_item',
                 'count_global_user_id_shop_id', 'positive_is_same_item_id', 'cvr_31_5_item_id', 'positive_visited_shop', 'cvr_31_5_shop_id',
                 'positive_is_same_shop_id', 'positive_seconds_gap', 'trades_31_5_item_id', 'ctr_item_id', 'ctr_shop_id',
                 'trades_31_5_shop_id', 'ctr_item_brand_id', 'item_pv_level_diff', 'property_num',
                 'count_global_user_id_item_price_level', 'count_item_id', 'counts_31_5_user_id',
                 'count_global_user_id', 'predict_category_item_category_ratio', 'count_global_user_id_shop_star_level', 
                 'count_global_item_id', 'count_user_id_shop_id', 'count_global_user_id_item_collected_level',
                 'trades_31_5_item_brand_id', 'count_item_brand_id', 'match_proetry_num', 'item_city_id',
                 'item_sales_level_diff', 'category_num', 'item_category_list1', 'item_brand_id', 'cvr_31_5_user_id',
                 'counts_31_5_user_id_item_id', 'context_page_id', 'count_user_id', 'count_shop_id', 'counts_31_5_shop_id',
                 'item_category_list2', 'item_collected_level_diff', 'average_user_id_item_id_visited_time_gap', 'ctr_user_id_shop_id',
                "apperance_nums_user_id_item_id_ratio","apperance_nums_user_id_ratio",
                 'itemid_0', 'itemid_1', 'itemid_2', 'itemid_3', 'itemid_4', 'itemid_5', 'itemid_6', 'itemid_7', 'itemid_8', 'itemid_9', 'itemid_10', 
                 'itemid_11', 'itemid_12', 'itemid_13', 'itemid_14', 'itemid_15', 'itemid_16', 'itemid_17', 'itemid_18', 'itemid_19',   
                ]

In [6]:
#构造数据集
train = df.loc[~np.isnan(df["is_trade"])]
test = df.loc[np.isnan(df["is_trade"])]

train["weight"] = 1
train.loc[(train["is_trade"]==0), 'weight'] = 1.2575195286787855


y = train['is_trade']
x = train
test = test

#for column in base_features:
#    x[column] = x[column].fillna(x[column].median())
#    test[column] = test[column].fillna(test[column].median())
#gc.collect()

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [7]:
def bagging_train(x_train,y_train,x_test):

    N_SPLITS=5

    print('x_train.shape=',x_train[base_features].shape,',y_train.shape=',y_train.shape)
    print('x_test.shape=',x_test[base_features].shape)
    kf=KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

    predict = np.zeros((len(x_test),1))
    
    for i,(train_index,test_index) in enumerate(kf.split(x_train,y_train)):
        x_tr=x_train.iloc[train_index]
        y_tr=y_train.iloc[train_index]
        tr_weight = x_train.iloc[train_index]["weight"]
        lgb_train = lgb.Dataset(x_tr[base_features], y_tr, weight=tr_weight)
        
        x_te=x_train.iloc[test_index]
        y_te=y_train.iloc[test_index]
        te_weight = x_train.iloc[test_index]["weight"]
        lgb_eval = lgb.Dataset(x_te[base_features], y_te, weight=te_weight)

        params = {
            'num_leaves': 63,
            'objective': 'binary',
            'min_data_in_leaf':50,
            'learning_rate': 0.01,
            'feature_fraction': 0.7,
            'bagging_fraction': 0.7,    
            'bagging_freq': 1,
            'max_depth':6,
            'metric': 'binary_logloss',    
        }

        gbm = lgb.train(params,
                        lgb_train,
                        num_boost_round=3000,
                        valid_sets=lgb_eval,
                        early_stopping_rounds=200)
        y_pred = gbm.predict(x_test[base_features], num_iteration=gbm.best_iteration)
        predict[:,0] = predict[:,0] + y_pred

    predict[:,0] = predict[:,0] / N_SPLITS
    return predict

In [8]:
predict = bagging_train(x,y,test)

x_train.shape= (1077175, 103) ,y_train.shape= (1077175,)
x_test.shape= (1729656, 103)
[1]	valid_0's binary_logloss: 0.684582
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's binary_logloss: 0.676186
[3]	valid_0's binary_logloss: 0.667954
[4]	valid_0's binary_logloss: 0.659878
[5]	valid_0's binary_logloss: 0.651956
[6]	valid_0's binary_logloss: 0.64419
[7]	valid_0's binary_logloss: 0.636574
[8]	valid_0's binary_logloss: 0.629101
[9]	valid_0's binary_logloss: 0.621759
[10]	valid_0's binary_logloss: 0.614556
[11]	valid_0's binary_logloss: 0.607489
[12]	valid_0's binary_logloss: 0.600552
[13]	valid_0's binary_logloss: 0.593738
[14]	valid_0's binary_logloss: 0.587049
[15]	valid_0's binary_logloss: 0.580479
[16]	valid_0's binary_logloss: 0.574031
[17]	valid_0's binary_logloss: 0.567697
[18]	valid_0's binary_logloss: 0.561473
[19]	valid_0's binary_logloss: 0.555355
[20]	valid_0's binary_logloss: 0.54934
[21]	valid_0's binary_logloss: 0.543431
[22]	valid_0's binary_

[201]	valid_0's binary_logloss: 0.179237
[202]	valid_0's binary_logloss: 0.178776
[203]	valid_0's binary_logloss: 0.178322
[204]	valid_0's binary_logloss: 0.177873
[205]	valid_0's binary_logloss: 0.177428
[206]	valid_0's binary_logloss: 0.176991
[207]	valid_0's binary_logloss: 0.176561
[208]	valid_0's binary_logloss: 0.176138
[209]	valid_0's binary_logloss: 0.175719
[210]	valid_0's binary_logloss: 0.175304
[211]	valid_0's binary_logloss: 0.174893
[212]	valid_0's binary_logloss: 0.174491
[213]	valid_0's binary_logloss: 0.174092
[214]	valid_0's binary_logloss: 0.1737
[215]	valid_0's binary_logloss: 0.173312
[216]	valid_0's binary_logloss: 0.172927
[217]	valid_0's binary_logloss: 0.172548
[218]	valid_0's binary_logloss: 0.172175
[219]	valid_0's binary_logloss: 0.171807
[220]	valid_0's binary_logloss: 0.171445
[221]	valid_0's binary_logloss: 0.171088
[222]	valid_0's binary_logloss: 0.170734
[223]	valid_0's binary_logloss: 0.170384
[224]	valid_0's binary_logloss: 0.170045
[225]	valid_0's bi

[402]	valid_0's binary_logloss: 0.147261
[403]	valid_0's binary_logloss: 0.147231
[404]	valid_0's binary_logloss: 0.147202
[405]	valid_0's binary_logloss: 0.14717
[406]	valid_0's binary_logloss: 0.147141
[407]	valid_0's binary_logloss: 0.147114
[408]	valid_0's binary_logloss: 0.147086
[409]	valid_0's binary_logloss: 0.147057
[410]	valid_0's binary_logloss: 0.147031
[411]	valid_0's binary_logloss: 0.147003
[412]	valid_0's binary_logloss: 0.146975
[413]	valid_0's binary_logloss: 0.146947
[414]	valid_0's binary_logloss: 0.14692
[415]	valid_0's binary_logloss: 0.146892
[416]	valid_0's binary_logloss: 0.146865
[417]	valid_0's binary_logloss: 0.146839
[418]	valid_0's binary_logloss: 0.146811
[419]	valid_0's binary_logloss: 0.146787
[420]	valid_0's binary_logloss: 0.146762
[421]	valid_0's binary_logloss: 0.146737
[422]	valid_0's binary_logloss: 0.146714
[423]	valid_0's binary_logloss: 0.146689
[424]	valid_0's binary_logloss: 0.146667
[425]	valid_0's binary_logloss: 0.146643
[426]	valid_0's bi

[603]	valid_0's binary_logloss: 0.144491
[604]	valid_0's binary_logloss: 0.144485
[605]	valid_0's binary_logloss: 0.14448
[606]	valid_0's binary_logloss: 0.144473
[607]	valid_0's binary_logloss: 0.144468
[608]	valid_0's binary_logloss: 0.14446
[609]	valid_0's binary_logloss: 0.144456
[610]	valid_0's binary_logloss: 0.144451
[611]	valid_0's binary_logloss: 0.144445
[612]	valid_0's binary_logloss: 0.144439
[613]	valid_0's binary_logloss: 0.144432
[614]	valid_0's binary_logloss: 0.144424
[615]	valid_0's binary_logloss: 0.144418
[616]	valid_0's binary_logloss: 0.144413
[617]	valid_0's binary_logloss: 0.144409
[618]	valid_0's binary_logloss: 0.144404
[619]	valid_0's binary_logloss: 0.144399
[620]	valid_0's binary_logloss: 0.144391
[621]	valid_0's binary_logloss: 0.144385
[622]	valid_0's binary_logloss: 0.144378
[623]	valid_0's binary_logloss: 0.144372
[624]	valid_0's binary_logloss: 0.144366
[625]	valid_0's binary_logloss: 0.14436
[626]	valid_0's binary_logloss: 0.144354
[627]	valid_0's bin

[804]	valid_0's binary_logloss: 0.143667
[805]	valid_0's binary_logloss: 0.143666
[806]	valid_0's binary_logloss: 0.143663
[807]	valid_0's binary_logloss: 0.143661
[808]	valid_0's binary_logloss: 0.143658
[809]	valid_0's binary_logloss: 0.143658
[810]	valid_0's binary_logloss: 0.143653
[811]	valid_0's binary_logloss: 0.14365
[812]	valid_0's binary_logloss: 0.143646
[813]	valid_0's binary_logloss: 0.143643
[814]	valid_0's binary_logloss: 0.14364
[815]	valid_0's binary_logloss: 0.143638
[816]	valid_0's binary_logloss: 0.143634
[817]	valid_0's binary_logloss: 0.143632
[818]	valid_0's binary_logloss: 0.143628
[819]	valid_0's binary_logloss: 0.143625
[820]	valid_0's binary_logloss: 0.143622
[821]	valid_0's binary_logloss: 0.143619
[822]	valid_0's binary_logloss: 0.143616
[823]	valid_0's binary_logloss: 0.143614
[824]	valid_0's binary_logloss: 0.143611
[825]	valid_0's binary_logloss: 0.14361
[826]	valid_0's binary_logloss: 0.143608
[827]	valid_0's binary_logloss: 0.143607
[828]	valid_0's bin

...

[2045]	valid_0's binary_logloss: 0.145807
[2046]	valid_0's binary_logloss: 0.145806
[2047]	valid_0's binary_logloss: 0.145807
[2048]	valid_0's binary_logloss: 0.145807
[2049]	valid_0's binary_logloss: 0.145806
[2050]	valid_0's binary_logloss: 0.145805
[2051]	valid_0's binary_logloss: 0.145805
[2052]	valid_0's binary_logloss: 0.145806
[2053]	valid_0's binary_logloss: 0.145805
[2054]	valid_0's binary_logloss: 0.145804
[2055]	valid_0's binary_logloss: 0.145804
[2056]	valid_0's binary_logloss: 0.145804
[2057]	valid_0's binary_logloss: 0.145804
[2058]	valid_0's binary_logloss: 0.145803
[2059]	valid_0's binary_logloss: 0.145803
[2060]	valid_0's binary_logloss: 0.145803
[2061]	valid_0's binary_logloss: 0.145802
[2062]	valid_0's binary_logloss: 0.145801
[2063]	valid_0's binary_logloss: 0.1458
[2064]	valid_0's binary_logloss: 0.1458
[2065]	valid_0's binary_logloss: 0.1458
[2066]	valid_0's binary_logloss: 0.145799
[2067]	valid_0's binary_logloss: 0.145799
[2068]	valid_0's binary_logloss: 0.1458


[2241]	valid_0's binary_logloss: 0.145756
[2242]	valid_0's binary_logloss: 0.145756
[2243]	valid_0's binary_logloss: 0.145756
[2244]	valid_0's binary_logloss: 0.145756
[2245]	valid_0's binary_logloss: 0.145756
[2246]	valid_0's binary_logloss: 0.145755
[2247]	valid_0's binary_logloss: 0.145755
[2248]	valid_0's binary_logloss: 0.145755
[2249]	valid_0's binary_logloss: 0.145755
[2250]	valid_0's binary_logloss: 0.145754
[2251]	valid_0's binary_logloss: 0.145754
[2252]	valid_0's binary_logloss: 0.145755
[2253]	valid_0's binary_logloss: 0.145754
[2254]	valid_0's binary_logloss: 0.145753
[2255]	valid_0's binary_logloss: 0.145753
[2256]	valid_0's binary_logloss: 0.145752
[2257]	valid_0's binary_logloss: 0.145752
[2258]	valid_0's binary_logloss: 0.145752
[2259]	valid_0's binary_logloss: 0.145751
[2260]	valid_0's binary_logloss: 0.14575
[2261]	valid_0's binary_logloss: 0.14575
[2262]	valid_0's binary_logloss: 0.145749
[2263]	valid_0's binary_logloss: 0.14575
[2264]	valid_0's binary_logloss: 0.14

[2437]	valid_0's binary_logloss: 0.145717
[2438]	valid_0's binary_logloss: 0.145717
[2439]	valid_0's binary_logloss: 0.145717
[2440]	valid_0's binary_logloss: 0.145717
[2441]	valid_0's binary_logloss: 0.145717
[2442]	valid_0's binary_logloss: 0.145717
[2443]	valid_0's binary_logloss: 0.145717
[2444]	valid_0's binary_logloss: 0.145717
[2445]	valid_0's binary_logloss: 0.145717
[2446]	valid_0's binary_logloss: 0.145716
[2447]	valid_0's binary_logloss: 0.145716
[2448]	valid_0's binary_logloss: 0.145715
[2449]	valid_0's binary_logloss: 0.145715
[2450]	valid_0's binary_logloss: 0.145717
[2451]	valid_0's binary_logloss: 0.145716
[2452]	valid_0's binary_logloss: 0.145716
[2453]	valid_0's binary_logloss: 0.145715
[2454]	valid_0's binary_logloss: 0.145715
[2455]	valid_0's binary_logloss: 0.145715
[2456]	valid_0's binary_logloss: 0.145714
[2457]	valid_0's binary_logloss: 0.145714
[2458]	valid_0's binary_logloss: 0.145714
[2459]	valid_0's binary_logloss: 0.145713
[2460]	valid_0's binary_logloss: 0

[2633]	valid_0's binary_logloss: 0.145692
[2634]	valid_0's binary_logloss: 0.145692
[2635]	valid_0's binary_logloss: 0.145691
[2636]	valid_0's binary_logloss: 0.14569
[2637]	valid_0's binary_logloss: 0.145691
[2638]	valid_0's binary_logloss: 0.14569
[2639]	valid_0's binary_logloss: 0.145689
[2640]	valid_0's binary_logloss: 0.145689
[2641]	valid_0's binary_logloss: 0.145689
[2642]	valid_0's binary_logloss: 0.145689
[2643]	valid_0's binary_logloss: 0.145689
[2644]	valid_0's binary_logloss: 0.145688
[2645]	valid_0's binary_logloss: 0.145688
[2646]	valid_0's binary_logloss: 0.145689
[2647]	valid_0's binary_logloss: 0.145688
[2648]	valid_0's binary_logloss: 0.145688
[2649]	valid_0's binary_logloss: 0.145688
[2650]	valid_0's binary_logloss: 0.145688
[2651]	valid_0's binary_logloss: 0.145688
[2652]	valid_0's binary_logloss: 0.145688
[2653]	valid_0's binary_logloss: 0.145688
[2654]	valid_0's binary_logloss: 0.145688
[2655]	valid_0's binary_logloss: 0.145687
[2656]	valid_0's binary_logloss: 0.1

[2829]	valid_0's binary_logloss: 0.14566
[2830]	valid_0's binary_logloss: 0.14566
[2831]	valid_0's binary_logloss: 0.14566
[2832]	valid_0's binary_logloss: 0.145659
[2833]	valid_0's binary_logloss: 0.145659
[2834]	valid_0's binary_logloss: 0.14566
[2835]	valid_0's binary_logloss: 0.145659
[2836]	valid_0's binary_logloss: 0.14566
[2837]	valid_0's binary_logloss: 0.14566
[2838]	valid_0's binary_logloss: 0.145659
[2839]	valid_0's binary_logloss: 0.145659
[2840]	valid_0's binary_logloss: 0.145658
[2841]	valid_0's binary_logloss: 0.145658
[2842]	valid_0's binary_logloss: 0.145658
[2843]	valid_0's binary_logloss: 0.145658
[2844]	valid_0's binary_logloss: 0.145658
[2845]	valid_0's binary_logloss: 0.145658
[2846]	valid_0's binary_logloss: 0.145658
[2847]	valid_0's binary_logloss: 0.145659
[2848]	valid_0's binary_logloss: 0.145659
[2849]	valid_0's binary_logloss: 0.145658
[2850]	valid_0's binary_logloss: 0.145659
[2851]	valid_0's binary_logloss: 0.145659
[2852]	valid_0's binary_logloss: 0.14565

In [ ]:
row_test = pd.read_csv("../../data/raw/round2_test_b.txt",sep=" ")
result = pd.DataFrame({"instance_id":row_test["instance_id"],"predicted_score":predict[-row_test.shape[0]:,0]})
result.to_csv("../../outputs/results.txt", sep=" ", index=False, encoding="utf-8",line_terminator="\n")